In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/m

In [9]:
from crossval import GridSearchCV
from implementations import logistic_regression
from proj1_helpers import load_csv_data, predict_labels, acc_score, f1_score


In [6]:
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
DATA_TEST_PATH = './data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [10]:
gridsearch = GridSearchCV(logistic_regression, predict_labels, f1_score,{'gamma':[0.01,0.1,1]} )

In [11]:
gridsearch.fit(y,tX)

{'gamma': 0.01} 0.37082699543671443
{'gamma': 0.1} 0.4191880010343936
{'gamma': 1} 0.41916515895580253


((('gamma', 0.1),),
 array([ 28.03219879,  -6.10752363,  -4.72826329,  -0.48841289,
         41.62864016,  43.16930885,  41.57038334,   0.85402851,
         -0.61629348,  -8.2066725 ,   0.83997905,   1.09139382,
         41.64361471,   0.11784366,   0.99967562,   0.99738463,
         -2.60465655,   1.00121915,   1.00016298,  -1.9066857 ,
          1.00504127, -11.68738052,   0.9253328 ,  35.18988532,
         37.26811001,  37.26810283,  39.99737631,  41.64486599,
         41.64319606,  -3.71985906]),
 15.714570804250107)